In [3]:
import scipy
import numpy as np

In [4]:
lambda_ = 0.1
n_factors = 100
m, n = 5000, 3000
n_iterations = 100

In [5]:
#M = np.random.randint(10, size=(m, n))
M = scipy.sparse.rand(m = m, n = n, density=0.01, format='coo', random_state=213)

X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)

def get_error(M, X, Y, W):
    return np.sum((W * (M - np.dot(X, Y)))**2)

W = np.ones(shape=(m, n))



In [ ]:
errors = []
for ii in range(n_iterations):
    X = np.linalg.solve(
          np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
          np.dot(Y, M.T)).T
    Y = np.linalg.solve(
           np.dot(X.T, X) + lambda_ * np.eye(n_factors),
           np.dot(X.T, M))
    err = get_error(M, X, Y, W)
    #if ii % 5 == 0:
    print('{}th iteration is completed,Error {}'.format(ii,err))
    errors.append(err)
M_hat = np.dot(X, Y)
print('Error: {}'.format(get_error(M, X, Y, W)))


In [ ]:
from pyspark.mllib.linalg import Matrix, Matrices
m,n = 30, 20

pop = int(m*n*0.1)

# ??? Not working
M1 = Matrices.sparse(
                     numRows = m, 
                     numCols = n, 
                     colPtrs = np.random.randint(n, size = pop), 
                     rowIndices = np.random.randint(m, size = pop), 
                     values = np.random.randint(20, size = pop)
)

X1 = Matrices.dense(m, n_factors, np.random.rand(m * n_factors))
Y1 = Matrices.dense(n_factors, n, np.random.rand(n * n_factors))


In [ ]:
help(Matrices.sparse)

In [ ]:
sm = Matrices.sparse(3, 4, [0, 1, 9, 2, 7], [0, 2, 1,1,2], [1, 2, 3,4,5])
sm.toArray()

In [8]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import time

def load_matrix(filename, num_users, num_items):
    t0 = time.time()
    counts = sparse.dok_matrix((num_users, num_items), dtype=float)
    total = 0.0
    num_zeros = num_users * num_items
    for i, line in enumerate(open(filename, 'r')):
        user, item, count = line.strip().split('\t')
        user = int(user)
        item = int(item)
        count = float(count)
        if user >= num_users:
            continue
        if item >= num_items:
            continue
        if count != 0:
            counts[user, item] = count
            total += count
            num_zeros -= 1
        if i % 100000 == 0:
            print 'loaded %i counts...' % i
    alpha = num_zeros / total
    print 'alpha %.2f' % alpha
    counts *= alpha
    counts = counts.tocsr()
    t1 = time.time()
    print 'Finished loading matrix in %f seconds' % (t1 - t0)
    return counts


class ImplicitMF():

    def __init__(self, counts, num_factors=40, num_iterations=30,
                 reg_param=0.8):
        self.counts = counts
        self.num_users = counts.shape[0]
        self.num_items = counts.shape[1]
        self.num_factors = num_factors
        self.num_iterations = num_iterations
        self.reg_param = reg_param

    def train_model(self):
        self.user_vectors = np.random.normal(size=(self.num_users,
                                                   self.num_factors))
        self.item_vectors = np.random.normal(size=(self.num_items,
                                                   self.num_factors))

        for i in xrange(self.num_iterations):
            t0 = time.time()
            print 'Solving for user vectors...'
            self.user_vectors = self.iteration(True, sparse.csr_matrix(self.item_vectors))
            print 'Solving for item vectors...'
            self.item_vectors = self.iteration(False, sparse.csr_matrix(self.user_vectors))
            t1 = time.time()
            print 'iteration %i finished in %f seconds' % (i + 1, t1 - t0)

    def iteration(self, user, fixed_vecs):
        num_solve = self.num_users if user else self.num_items
        num_fixed = fixed_vecs.shape[0]
        YTY = fixed_vecs.T.dot(fixed_vecs)
        eye = sparse.eye(num_fixed)
        lambda_eye = self.reg_param * sparse.eye(self.num_factors)
        solve_vecs = np.zeros((num_solve, self.num_factors))

        t = time.time()
        for i in xrange(num_solve):
            if user:
                counts_i = self.counts[i].toarray()
            else:
                counts_i = self.counts[:, i].T.toarray()
            CuI = sparse.diags(counts_i, [0])
            pu = counts_i.copy()
            pu[np.where(pu != 0)] = 1.0
            YTCuIY = fixed_vecs.T.dot(CuI).dot(fixed_vecs)
            YTCupu = fixed_vecs.T.dot(CuI + eye).dot(sparse.csr_matrix(pu).T)
            xu = spsolve(YTY + YTCuIY + lambda_eye, YTCupu)
            solve_vecs[i] = xu
            if i % 1000 == 0:
                print 'Solved %i vecs in %d seconds' % (i, time.time() - t)
                t = time.time()

        return solve_vecs

In [11]:
a = ImplicitMF(counts = M)

a.train_model()

Solving for user vectors...


TypeError: 'coo_matrix' object does not support indexing